In [1]:
# Import libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import PIL.Image as Image
from IPython.display import display
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
dataset_dir_train = "../input/training-dataset-v6/finalDataset"

train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                 transforms.RandomRotation(20),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


dataset = torchvision.datasets.ImageFolder(root=dataset_dir_train, transform = train_transform)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, num_workers = 2)

In [4]:
# function to train the model
def TrainModel(model, lossFun, optimizer, scheduler, n_epochs=5):
    losses = []
    accuracies = []
    
    model.train()
    
    for epoch in range(n_epochs):
        print(epoch)
        since = time.time() # record the start time
        running_loss = 0.0
        running_correct = 0.0
        counter = 0
        for i, data in enumerate(trainloader, 0):
            
            inputs, labels = data
            # Asigning GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            counter += len(inputs)
            outputs = model(inputs)
            m = nn.LogSoftmax(dim=1)
            _, predicted = torch.max(outputs.data, 1)
            loss = lossFun(m(outputs), labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        timeSpent = time.time()-since # time used
        epoch_loss = running_loss/counter
        print(running_correct, counter)
        epoch_acc = 100*(running_correct/counter)
        print("Epoch %s, Time Used: %d s, loss: %.4f, acc: %.4f" % (epoch+1, timeSpent, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        scheduler.step(epoch_acc)
    print('Training Finished')
    return model, losses, accuracies

In [5]:
class EnsembleModel(nn.Module):   
    def __init__(self, modelA, modelB, modelC):
        super().__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        self.classifier = nn.Linear(23 * 3, 23)
        
    def forward(self, x):
        x1 = self.modelA(x)
        x2 = self.modelB(x)
        x3 = self.modelC(x)
        x = torch.cat((x1, x2, x3), dim=1)
        out = self.classifier(x)
        return out

In [6]:
regnet_v6 = torch.load("../input/models/regnet_y_3_2gf_v6.pt", torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
regnet_v4 = torch.load("../input/models/regnet_y_3_2gf_v4.pt", torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
regnet_1000x400 = torch.load("../input/models/regnet_y_3_2gf_v(1000 x 400 x 23).pt", torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

In [7]:
ensemble_model = EnsembleModel(regnet_v6, regnet_v4, regnet_1000x400)

for param in ensemble_model.parameters():
    param.requires_grad = False

for param in ensemble_model.classifier.parameters():
    param.requires_grad = True    

ensemble_model = ensemble_model.to("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
lossFun = nn.NLLLoss()
optimizer = optim.SGD(ensemble_model.parameters(), lr=0.007, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, threshold = 0.9)

In [9]:
model, training_losses, training_accs = TrainModel(ensemble_model, lossFun, optimizer, lrscheduler, n_epochs=25)

0
45130.0 46374
Epoch 1, Time Used: 589 s, loss: 0.0031, acc: 97.3175
1
45491.0 46374
Epoch 2, Time Used: 526 s, loss: 0.0022, acc: 98.0959
2
45536.0 46374
Epoch 3, Time Used: 525 s, loss: 0.0021, acc: 98.1930
3
45537.0 46374
Epoch 4, Time Used: 523 s, loss: 0.0021, acc: 98.1951
4
45740.0 46374
Epoch 5, Time Used: 522 s, loss: 0.0013, acc: 98.6329
5
45760.0 46374
Epoch 6, Time Used: 519 s, loss: 0.0012, acc: 98.6760
6
45745.0 46374
Epoch 7, Time Used: 522 s, loss: 0.0011, acc: 98.6436
7
45787.0 46374
Epoch 8, Time Used: 523 s, loss: 0.0010, acc: 98.7342
8
45751.0 46374
Epoch 9, Time Used: 523 s, loss: 0.0011, acc: 98.6566
9
45753.0 46374
Epoch 10, Time Used: 521 s, loss: 0.0011, acc: 98.6609
10
45780.0 46374
Epoch 11, Time Used: 522 s, loss: 0.0011, acc: 98.7191
11
45769.0 46374
Epoch 12, Time Used: 520 s, loss: 0.0010, acc: 98.6954
12
45794.0 46374
Epoch 13, Time Used: 522 s, loss: 0.0011, acc: 98.7493
13
45762.0 46374
Epoch 14, Time Used: 525 s, loss: 0.0011, acc: 98.6803
14
45767.0 

In [10]:
model.eval()
torch.save(model, "file.pt")